## Gather

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
patients = pd.read_csv('patients.csv')
treatments = pd.read_csv('treatments.csv')
adverse_reactions = pd.read_csv('adverse_reactions.csv')

## Assess

In [ ]:
patients

In [ ]:
treatments

In [ ]:
adverse_reactions

In [ ]:
patients.info()

In [ ]:
treatments.info()

In [ ]:
adverse_reactions.info()

In [ ]:
all_columns = pd.Series(list(patients) + list(treatments) + list(adverse_reactions))
all_columns[all_columns.duplicated()]

In [ ]:
list(patients)

In [ ]:
patients[patients['address'].isnull()]

In [ ]:
patients.describe()

In [ ]:
treatments.describe()

In [ ]:
patients.sample(5)

In [ ]:
patients.surname.value_counts()

In [ ]:
patients.address.value_counts()

In [ ]:
patients[patients.address.duplicated()]

In [ ]:
patients.weight.sort_values()

In [ ]:
weight_lbs = patients[patients.surname == 'Zaitseva'].weight * 2.20462
height_in = patients[patients.surname == 'Zaitseva'].height
bmi_check = 703 * weight_lbs / (height_in * height_in)
bmi_check

In [ ]:
patients[patients.surname == 'Zaitseva'].bmi

In [ ]:
sum(treatments.auralin.isnull())

In [ ]:
sum(treatments.novodra.isnull())

#### Quality
##### `patients` table
- Zip code is a float not a string
- Zip code has four digits sometimes
- Tim Neudorf height is 27 in instead of 72 in
- Full state names sometimes, abbreviations other times
- Dsvid Gustafsson
- Missing demographic information (address - contact columns) ***(can't clean)***
- Erroneous datatypes (assigned sex, state, zip_code, and birthdate columns)
- Multiple phone number formats
- Default John Doe data
- Multiple records for Jakobsen, Gersten, Taylor
- kgs instead of lbs for Zaitseva weight

##### `treatments` table
- Missing HbA1c changes
- The letter 'u' in starting and ending doses for Auralin and Novodra
- Lowercase given names and surnames
- Missing records (280 instead of 350)
- Erroneous datatypes (auralin and novodra columns)
- Inaccurate HbA1c changes (leading 4s mistaken as 9s)
- Nulls represented as dashes (-) in auralin and novodra columns

##### `adverse_reactions` table
- Lowercase given names and surnames

#### Tidiness
- Contact column in `patients` table should be split into phone number and email
- Three variables in two columns in `treatments` table (treatment, start dose and end dose)
- Adverse reaction should be part of the `treatments` table
- Given name and surname columns in `patients` table duplicated in `treatments` and `adverse_reactions` tables

## Clean

In [ ]:
patients_clean = patients.copy()
treatments_clean = treatments.copy()
adverse_reactions_clean = adverse_reactions.copy()

### Missing Data

#### `treatments`: Missing records (280 instead of 350)

##### Define
Use `pd.concat` to concatonate the cut dataset to the treatments dataframe

Note: the missing `treatments` records are stored in a file named `treatments_cut.csv`, which you can see in this Jupyter Notebook's dashboard (click the **jupyter** logo in the top lefthand corner of this Notebook). Hint: [documentation page](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.concat.html) for the function used in the solution.*

##### Code

In [ ]:
treat_cuts = pd.read_csv('treatments_cut.csv')
treatments_clean = pd.concat([treatments_clean, treat_cuts], ignore_index = True);

##### Test

In [ ]:
treatments_clean.head()

In [ ]:
treatments_clean.tail()

#### `treatments`: Missing HbA1c changes and Inaccurate HbA1c changes (leading 4s mistaken as 9s)
*Note: the "Inaccurate HbA1c changes (leading 4s mistaken as 9s)" observation, which is an accuracy issue and not a completeness issue, is included in this header because it is also fixed by the cleaning operation that fixes the missing "Missing HbA1c changes" observation. Multiple observations in one **Define, Code, and Test** header occurs multiple times in this notebook.*

##### Define
Assign change in HbA1c to the difference between start and finish levels of HbA1c

##### Code

In [ ]:
# Your cleaning code here
treatments_clean['hba1c_change'] = treatments_clean['hba1c_start'] - treatments_clean['hba1c_end']

##### Test

In [ ]:
# Your testing code here
treatments_clean.head()

In [ ]:
sum(treatments_clean.hba1c_change.isnull())

### Tidiness

#### Contact column in `patients` table contains two variables: phone number and email

##### Define
Use `str.extract` method to extract phone number and email patterns and store them in patients_clean columns


##### Code

In [ ]:
patients_clean['phone_number'] = patients_clean.contact.str.extract(r'((\+\d{1,2}\s)?\(?\d{3}\)?[\s.-]\d{3}[\s.-]\d{4})', expand = True)[0]

#all email addresses in dataset start and end with letters
patients_clean['email'] = patients_clean.contact.str.extract(r'([a-zA-Z][a-zA-Z0-9_.+-]+@[a-zA-Z0-9-]+\.[a-zA-Z0-9-.]+[a-zA-Z]+)', expand = True)

#drop faulty column
patients_clean = patients_clean.drop('contact', axis = 1)

##### Test

In [ ]:
list(patients_clean)

In [ ]:
patients_clean.sample(25);

#### Three variables in two columns in `treatments` table (treatment, start dose and end dose)

##### Define
Split start dose, end dose, and treatment into three columns. 
Split auralin column into start and end dose columns

*Your definition here. Hint: use pandas' [melt function](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.melt.html) and [`str.split()` method](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.str.split.html). Here is an excellent [`melt` tutorial](https://deparkes.co.uk/2016/10/28/reshape-pandas-data-with-melt/).*

##### Code

In [ ]:
# Melt df so that auralin and novodra are their own variable.
treatments_clean = pd.melt(treatments_clean, id_vars=['given_name', 'surname', 'hba1c_start', 'hba1c_end', 'hba1c_change'], var_name='treatment', value_name='dose')

In [ ]:
#observations are now doubled, with an entry for every patient put in twice, the only difference is treatment and dose
treatments_clean = treatments_clean[treatments_clean.dose != '-']
#puts dose splits into vectorized string function
treatments_clean['start_dose'], treatments_clean['end_dose'] = treatments_clean['dose'].str.split(' - ', 1).str;

In [ ]:
#drop intermediate 'dose'
treatments_clean.drop('dose', axis = 1, inplace = True)

##### Test

In [ ]:
treatments_clean.head()

In [ ]:
treatments_clean.tail()

#### Adverse reaction should be part of the `treatments` table

##### Define
Merge adverse reactions to treatment tables along rows

##### Code

In [ ]:
# Your cleaning code here
treatments_clean2 = pd.merge(treatments_clean, adverse_reactions_clean, on = ['given_name', 'surname'], how = 'outer')

##### Test

In [ ]:
# Your testing code here
treatments_clean2

#### Given name and surname columns in `patients` table duplicated in `treatments` and `adverse_reactions` tables  and Lowercase given names and surnames

##### Define
lower given name and surname and merge datasets between patients and treatments

##### Code

In [ ]:
# Your cleaning code here
patients_clean['given_name'] = patients_clean.given_name.str.lower()
patients_clean['surname'] = patients_clean.surname.str.lower()

In [ ]:
patients_clean2 = pd.merge(treatments_clean2, patients_clean, on = ['given_name', 'surname'], how = 'left')
patients_clean2

##### Test

In [ ]:
patients_clean2[patients_clean2.duplicated() == True]

### Quality

#### Zip code is a float not a string and Zip code has four digits sometimes

##### Define
convert zipcode to string using `astype` and pad code with zeroes

##### Code

In [ ]:
# Your cleaning code here
patients_clean2['zip_code'] = patients_clean2.zip_code.astype(str).str[:-2].str.pad(width = 5, side = 'left', fillchar = '0')
#This replaces some zipcodes to 0000n, so they must be converted back
patients_clean2.zip_code = patients_clean2.zip_code.replace('0000n', np.nan)

##### Test

In [ ]:
# Your testing code here
patients_clean2[patients_clean2['zip_code'] == '0000n']

#### Tim Neudorf height is 27 in instead of 72 in

##### Define
Edit height of Tim Neudorf using replace. Change 27 to 72

##### Code

In [ ]:
patients_clean2.height.value_counts();
#There is only one instance of 27
patients_clean2.height = patients_clean2.height = patients_clean2.height.replace(27, 72)

##### Test

In [ ]:
# Your testing code here
patients_clean[patients_clean.surname == 'neudorf']

#### Full state names sometimes, abbreviations other times

##### Define
*Your definition here. Hint: [tutorial](https://chrisalbon.com/python/pandas_apply_operations_to_dataframes.html) for method used in solution.*

##### Code

In [ ]:
# Your cleaning code here
patients_clean2.state.value_counts()
patients_clean2.state = patients_clean2.state.replace(['New York', 'California', 'Illinois', 'Florida', 'Nebraska'], ['NY', 'CA', 'IL', 'FL', 'NE'])

##### Test

In [ ]:
# Your testing code here
patients_clean2.state.value_counts()

#### Dsvid Gustafsson

##### Define
Replace Dsvid Gustafsson with David

##### Code

In [ ]:
# Your cleaning code here
patients_clean2.given_name = patients_clean2.given_name.replace('dsvid', 'david')

##### Test

In [ ]:
# Your testing code here
patients_clean2[patients_clean2.surname == 'gustafsson']

#### Erroneous datatypes (assigned sex, state, zip_code, and birthdate columns) and Erroneous datatypes (auralin and novodra columns) and The letter 'u' in starting and ending doses for Auralin and Novodra

##### Define
Use strip to strip "u's" on start dose and end dose
use date time on birthdate columns
use astype on assigned sex and state

*Your definition here. Hint: [documentation page](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.astype.html) for one method used in solution, [documentation page](http://pandas.pydata.org/pandas-docs/version/0.20/generated/pandas.to_datetime.html) for one function used in the solution, and [documentation page](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.Series.str.strip.html) for another method used in the solution.*

##### Code

In [ ]:
# Your cleaning code here
#to category
patients_clean2.assigned_sex = patients.assigned_sex.astype('category')
patients_clean2.state = patients_clean2.state.astype('category')

In [ ]:
#to datetime
patients_clean2.birthdate = pd.to_datetime(patients_clean.birthdate, infer_datetime_format = True)

In [ ]:
patients_clean2

In [ ]:
#strip u's
patients_clean2.start_dose.str.strip('u').astype(int)
patients_clean2.end_dose.str.strip('u').astype(int)

##### Test

In [ ]:
# Your testing code here

#### Multiple phone number formats

##### Define
*Your definition here. Hint: helpful [Stack Overflow answer](https://stackoverflow.com/a/123681).*

##### Code

In [ ]:
# Your cleaning code here

##### Test

In [ ]:
# Your testing code here

#### Default John Doe data

##### Define
*Your definition here. Recall that it is assumed that the data that this John Doe data displaced is not recoverable.*

##### Code

In [ ]:
# Your cleaning code here

##### Test

In [ ]:
# Your testing code here

#### Multiple records for Jakobsen, Gersten, Taylor

##### Define
*Your definition here.*

##### Code

In [ ]:
# Your cleaning code here

##### Test

In [ ]:
# Your testing code here

#### kgs instead of lbs for Zaitseva weight

##### Define
*Your definition here.*

##### Code

In [ ]:
# Your cleaning code here

##### Test

In [ ]:
# Your testing code here